In [3]:
import pandas as pd
import numpy as np
import holoviews as hv
from math import pi
import plotly.offline as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


init_notebook_mode(connected=True)

In [4]:
data = pd.read_csv('/Users/michael.chestnut/Documents/DOI/data/cleaned_data.csv')

In [5]:
df = data.drop(columns='Unnamed: 0')
df.head(5)

,UII,bureau_name,portfolio_part,investment_title,fea_service_area,FY18,FY19,FY20,date
0,010-000002256,Department of the Interior,IT,IBC - Infrastructure - IT Security & Compliance,315 - Threat and Vulnerability Management,0.000000,0.000000,0.000000,2019-04-03
1,010-000002378,Department of the Interior,Admin Services,DOI - FOIA Portal (DOJ),319 - Regulatory Compliance,0.000000,0.030000,0.030000,2019-04-03
2,010-000001448,Department of the Interior,Mission Delivery,OS - Receipts Expenditures and Allocation Mode...,101 - Budget Formulation,0.095000,0.103000,0.101000,2019-04-03
3,010-000000310,Department of the Interior,Admin Services,DOI - Enterprise Facilities Maintenance & Mana...,129 - Reporting and Information,5.175915,5.175962,5.259022,2019-04-03
4,010-000002321,Department of the Interior,Admin Services,DOI - Compliance Support ESF-11/ESF-11 Website,008 - Disaster Preparedness and Planning,0.090902,0.106702,0.090902,2019-04-03


In [6]:
df['trimmed_investment_title'] = df['investment_title'].str.split('-',1).str[1]
df.head(5)

,UII,bureau_name,portfolio_part,investment_title,fea_service_area,FY18,FY19,FY20,date,trimmed_investment_title
0,010-000002256,Department of the Interior,IT,IBC - Infrastructure - IT Security & Compliance,315 - Threat and Vulnerability Management,0.000000,0.000000,0.000000,2019-04-03,Infrastructure - IT Security & Compliance
1,010-000002378,Department of the Interior,Admin Services,DOI - FOIA Portal (DOJ),319 - Regulatory Compliance,0.000000,0.030000,0.030000,2019-04-03,FOIA Portal (DOJ)
2,010-000001448,Department of the Interior,Mission Delivery,OS - Receipts Expenditures and Allocation Mode...,101 - Budget Formulation,0.095000,0.103000,0.101000,2019-04-03,Receipts Expenditures and Allocation Model (R...
3,010-000000310,Department of the Interior,Admin Services,DOI - Enterprise Facilities Maintenance & Mana...,129 - Reporting and Information,5.175915,5.175962,5.259022,2019-04-03,Enterprise Facilities Maintenance & Managemen...
4,010-000002321,Department of the Interior,Admin Services,DOI - Compliance Support ESF-11/ESF-11 Website,008 - Disaster Preparedness and Planning,0.090902,0.106702,0.090902,2019-04-03,Compliance Support ESF-11/ESF-11 Website


In [7]:
def assign_colors(portfolio):
    if portfolio == 'IT':
        return '#8dd3c7'
    elif portfolio == 'Mission Delivery':
        return '#ffffb3'
    else:
        return '#bebada'

In [9]:
def update_bars(bureau='Department of the Interior',fy='FY18'):
    filtered_df = df[['bureau_name','portfolio_part','trimmed_investment_title',fy]]
    filtered_df = filtered_df[filtered_df['bureau_name']==bureau]
    
    #Sort and return top 10 based on FY values
    top10 = filtered_df.nlargest(10,fy)
    top10['color'] = top10['portfolio_part'].apply(assign_colors)
    
    #Fill in chart data and layout information
    data=[]
    for i in range(0, len(pd.unique(top10['portfolio_part']))):
        data.append(go.Bar(
            x= top10[fy][top10['portfolio_part'] == pd.unique(top10['portfolio_part'])[i]],
            y= top10['trimmed_investment_title'][top10['portfolio_part'] == pd.unique(top10['portfolio_part'])[i]],
            name= pd.unique(top10['portfolio_part'])[i],
            orientation= 'h',
            marker=dict(
                color = top10['color'][top10['portfolio_part'] == pd.unique(top10['portfolio_part'])[i]]
            )   
        ))
            
    layout = go.Layout(
        margin = dict(l = 400, r = 50, b = 50, t = 50, pad = 4),
        xaxis=dict(
            title=str(fy) +' Spending (In Millions)'),
        showlegend=True
    )
    
    fig = go.Figure(data=data,layout=layout)
    ply.iplot(fig, filename='horizontal-bar')
    
update_bars()